In [1]:
import os
import pydicom
import highdicom as hd
import pandas as pd
import numpy as np
import dicom2nifti

In [12]:
#Extract SOP IDS from segmentation object
dirPath = '.'
count_seg = 0
SOP = []

In [23]:
for studies in os.listdir(dirPath):
    if('CRLM-CT' in studies):
        study_contents = os.path.join(dirPath,studies)
        for segment in os.listdir(study_contents):
            if 'seg' in segment:
                segPath = os.path.join(study_contents,segment)
                seg = hd.seg.segread(segPath)
                segment_uids = pd.DataFrame(seg.get_source_image_uids())
                sop_ids = segment_uids.iloc[:,-1:].to_numpy()
                
                outPath2 = os.path.join(study_contents, 'dicom_liver')
                if os.path.exists(outPath2):
                    print('path to liver dicom already exists')
                else:
                    os.mkdir(outPath2)
                    
                file_list = []
                #make dicom path
                for dicom in os.listdir(study_contents):
                    if 'dicom' in dicom:
                        dicomPath = os.path.join(study_contents,dicom)
                        for files in os.listdir(dicomPath):
                            filename = os.path.join(dicomPath, files)
                            ds = pydicom.filereader.dcmread(filename)
                            if(np.isin(ds.SOPInstanceUID, sop_ids)):
                                file_list.append(filename)
                                os.rename(filename, os.path.join(outPath2, files))

In [24]:
#write nii.gz files for each segmentation
os.mkdir('./LiverSegmentations')

In [29]:
count = 0
outPath = './LiverSegmentations'
for studies in os.listdir(dirPath):
    if('CRLM-CT' in studies):
        study_contents = os.path.join(dirPath,studies)
        for segment in os.listdir(study_contents):
            if 'dicom_liver' in segment:
                dicomLiverPath = os.path.join(study_contents, segment)
                outLiverPath = os.path.join(outPath, studies)
                #os.mkdir(outLiverPath)
                
                outLiverFile = os.path.join(outLiverPath, 'liver.nii.gz')
                dicom2nifti.dicom_series_to_nifti(dicomLiverPath, outLiverFile, reorient_nifti=True)
                count+=1

print(count)
        

197
